In [1]:
#BASED ON THE WORK OF Sahingoz, Ozgur K.; Buber, Ebubekir; Demir, Onder; Diri, Banu
#ARTICLE: Machine Learning Based Phishing Detection from URLs

import numpy as np

In [2]:
#FUNÇÃO PARA SEPARAR POR MÚLTIPLOS CARACTERES

def tsplit(string, delimiters):
    """Behaves str.split but supports multiple delimiters."""
    
    delimiters = tuple(delimiters)
    stack = [string,]
    
    for delimiter in delimiters:
        for i, substring in enumerate(stack):
            substack = substring.split(delimiter)
            stack.pop(i)
            for j, _substring in enumerate(substack):
                stack.insert(i+j, _substring)
            
    return stack

caracteres = ["[", " ", ":", "`", "\\", "-", "=", "~", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "+", "\\\\", "[", "]", "{", "}", ";", "'", "\\:", "|", "<", ",", ".", "/", "<", ">", "?", "]"]

In [3]:
url = "http://www.crjewelers.net/crjguest/"

lista = tsplit(url, caracteres)

#FUNÇÃO DE REMOVER CARACTERES ADICIONAIS NA URL
def remover_adicionais(i, lista):
    if i == '':
        lista.remove('')
    elif i == "http":
        lista.remove("http")
    elif i == "www":
        lista.remove("www")
    elif i == "\n":
        lista.remove("\n")

for i in lista[:]:
    remover_adicionais(i, lista)

print(lista)

['crjewelers', 'net', 'crjguest']


In [4]:
#PEGANDO O TXT DE URLS_INVALIDOS

f = open("urls_invalidos.txt", encoding="utf8")

txt = f.readlines()

print(txt[4])

http://www.cosmeticasaludable.com/



In [5]:
#TESTANDO O SPLIT NOS URLS INVALIDOS DO TXT

invalidos = []

for i in txt[:5]:
    invalidosAux = tsplit(i, caracteres)
    for j in invalidosAux[:]:
        remover_adicionais(j, invalidosAux)
    invalidos.append(invalidosAux)
    
print(invalidos)
print(len(invalidos))
print(len(invalidos[1]))
print(invalidos[1])

[['crjewelers', 'net', 'crjguest'], ['b2b', 'marketplace', 'opensourcescripts', 'com'], ['changtsi', 'com', 'js', 'newlogin', 'php\n'], ['asterasepe', 'gr'], ['cosmeticasaludable', 'com']]
5
4
['b2b', 'marketplace', 'opensourcescripts', 'com']


In [6]:
#TESTANDO COM A URL EXEMPLO

example = "www.changtsi.com/weuyrwbr/"
listaExample = tsplit(example, caracteres)

for i in listaExample:
    remover_adicionais(i, listaExample)

print(listaExample)
print(listaExample[1])

['changtsi', 'com', 'weuyrwbr']
com


In [7]:
def acharIgual(palavra, invalidos):
    i = 0
    j = 0
    while i < len(invalidos):
        while j < len(invalidos[i]):
            if palavra == invalidos[i][j]:
                return palavra
            j += 1
        i += 1
    return "não tem"

In [8]:
#TESTANDO PARA VER SE ACHA ALGUMA PALAVRA JÁ SEPARADA DA URL EXEMPLO QUE SEJA IGUAL A ALGUMA DO TXT DE INVÁLIDOS

aux = acharIgual(listaExample[0], invalidos)
print(aux)

não tem


In [9]:
#TESTANDO COM A URL EXEMPLO FINAL PROS OUTROS TESTES

exemplo = "www.adidas.com/activision-blizzard/lorota1/crjguest/banking/wfbuwnuernfuefberyu"

exLista = tsplit(exemplo, caracteres)
for i in exLista:
    remover_adicionais(i, exLista)

print(exLista)

['adidas', 'com', 'activision', 'blizzard', 'lorota1', 'crjguest', 'banking', 'wfbuwnuernfuefberyu']


In [10]:
#ABRINDO O TXT DAS MARCAS

b = open("allbrands.txt", "r", encoding="utf8")

allbrands = b.read()
allbrandsList = allbrands.split("\n")

print(allbrandsList[2])

adidas


In [11]:
#ABRINDO O TXT DAS KEYWORDS

c = open("keywords.txt", "r", encoding="utf8")

keywords = c.read()
keywordsList = keywords.split("\n")

print(keywordsList[2])

access


In [12]:
#NO MOMENTO ESTÁ SÓ REMOVENDO DA LISTA, MAS NA PIPELINE INFORMA QUE AS PALAVRAS QUE FOREM IGUAIS A ALGUMA DOS ARQUIVOS
#VAI PARA A FEATURE EXTRACTION

def achar_igual(palavra, lista):
    for i in lista:
        if i == palavra:
            return palavra
    return "não tem"

def achar_existentes(exLista, documento):
    for i in exLista:
        resposta = achar_igual(i, documento)
        if resposta != "não tem":
            exLista.remove(resposta)


In [13]:
#MÓDULO "BRAND NAMES AND KEYWORDS CHECK"

achar_existentes(exLista, allbrandsList)
achar_existentes(exLista, keywordsList)

print(exLista)

['activision', 'blizzard', 'lorota1', 'crjguest', 'wfbuwnuernfuefberyu']


In [14]:
#ALGORITMO PARA DETECTAR PALAVRAS ALEATORIAS

import math
import pickle

accepted_chars = 'abcdefghijklmnopqrstuvwxyz '

pos = dict([(char, idx) for idx, char in enumerate(accepted_chars)])

def normalize(line):
    """ Return only the subset of chars from accepted_chars.
    This helps keep the  model relatively small by ignoring punctuation, 
    infrequenty symbols, etc. """
    return [c.lower() for c in line if c.lower() in accepted_chars]

def ngram(n, l):
    """ Return all n grams from l after normalizing """
    filtered = normalize(l)
    for start in range(0, len(filtered) - n + 1):
        yield ''.join(filtered[start:start + n])

def train():
    """ Write a simple model as a pickle file """
    k = len(accepted_chars)
    # Assume we have seen 10 of each character pair.  This acts as a kind of
    # prior or smoothing factor.  This way, if we see a character transition
    # live that we've never observed in the past, we won't assume the entire
    # string has 0 probability.
    counts = [[10 for i in range(k)] for i in range(k)]

    # Count transitions from big text file, taken 
    # from http://norvig.com/spell-correct.html
    for line in open('big.txt'):
        for a, b in ngram(2, line):
            counts[pos[a]][pos[b]] += 1

    # Normalize the counts so that they become log probabilities.  
    # We use log probabilities rather than straight probabilities to avoid
    # numeric underflow issues with long texts.
    # This contains a justification:
    # http://squarecog.wordpress.com/2009/01/10/dealing-with-underflow-in-joint-probability-calculations/
    for i, row in enumerate(counts):
        s = float(sum(row))
        for j in range(len(row)):
            row[j] = math.log(row[j] / s)

    # Find the probability of generating a few arbitrarily choosen good and
    # bad phrases.
    good_probs = [avg_transition_prob(l, counts) for l in open('good.txt')]
    bad_probs = [avg_transition_prob(l, counts) for l in open('bad.txt')]

    # Assert that we actually are capable of detecting the junk.
    assert min(good_probs) > max(bad_probs)

    # And pick a threshold halfway between the worst good and best bad inputs.
    thresh = (min(good_probs) + max(bad_probs)) / 2
    pickle.dump({'mat': counts, 'thresh': thresh}, open('gib_model.pki', 'wb'))

def avg_transition_prob(l, log_prob_mat):
    """ Return the average transition prob from l through log_prob_mat. """
    log_prob = 0.0
    transition_ct = 0
    for a, b in ngram(2, l):
        log_prob += log_prob_mat[pos[a]][pos[b]]
        transition_ct += 1
    # The exponentiation translates from log probs to probs.
    return math.exp(log_prob / (transition_ct or 1))

if __name__ == '__main__':
    train()

In [15]:
#RANDOM WORD DETECTION
#TAMBÉM ESTÁ SÓ REMOVENDO, MAS DEVE IR PARA FEATURE SELECTION

model_data = pickle.load(open('gib_model.pki', 'rb'))

for i in exLista[:]:
    l = i
    model_mat = model_data['mat']
    threshold = model_data['thresh']
    naoAleatorio = avg_transition_prob(l, model_mat) > threshold
    if naoAleatorio == False:
        exLista.remove(i)
        
print(exLista)

['activision', 'blizzard', 'lorota1']


In [16]:
#IMPORTANDO A BIBLIOTECA DE DICIONARIO (SO DISPONIVEL PARA LINUX)

import enchant

disc = enchant.Dict("en_US")
enchant.list_languages()

['en', 'en_AU', 'en_CA', 'en_GB', 'en_US']

In [17]:
#Remoção de dígitos
from string import digits 
  
def remove(list): 
    list = [''.join(x for x in i if x.isalpha()) for i in list] 
              
    return list

remove(exLista)
    
print(exLista)

['activision', 'blizzard', 'lorota1']


In [18]:
#checar existente no dicionario e menores que 3

for i in exLista[:]:
    if (disc.check(i) == True or len(i) <= 3):
        exLista.remove(i)
        
print(exLista)

['activision', 'lorota1']


In [19]:
#dividir em substrings e comparar
#não leva em consideração só as palavras com tamanho maior que 7, como no artigo original

from nltk import everygrams

substrings = []

for i in exLista[:]:
    word = i
    subs = [''.join(_ngram) for _ngram in everygrams(word) if disc.check(''.join(_ngram)) and len(_ngram) > 1]
    substrings.append(subs)

print(substrings)

for i in substrings:
    for j in i:
        if disc.check(j) == True:
            print("proceed :>")
        else:
            print("doesn't proceed :/")

[['ac', 'ct', 'ti', 'iv', 'vi', 'is', 'on', 'act', 'ion', 'vision'], ['lo', 'or', 'ta', 'rot', 'rota']]
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>
proceed :>


In [20]:
#COMPARANDO SUBSTRINGS COM PALAVRAS NOS ARQUIVOS

for i in substrings:
    achar_existentes(i, allbrandsList)
    achar_existentes(i, keywordsList)
    
print(substrings)

[['ac', 'ct', 'ti', 'iv', 'vi', 'is', 'on', 'act', 'ion', 'vision'], ['lo', 'or', 'ta', 'rot', 'rota']]


In [21]:
#ALGORITMO LEVENSHTEIN DISTANCE

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

In [22]:
#SEPARA OS VALORES MENORES DE 2
#ESTÁ REMOVENDO OS ACUSADOS PELO LEVENSHTEIN DISTANCE, MAS DEVEM IR PARA FEATURE EXTRACTION

def calcular_distancia(palavra, lista):
    for i in lista:
        distan = levenshtein(palavra, i)
        if distan < 2:
            return "procede"
    return "nao procede"

for i in substrings:
    for j in i:
        resp1 = calcular_distancia(j, allbrandsList)
        resp2 = calcular_distancia(j, keywordsList)
        
        if (resp1 == "procede" or resp2 == "procede"):
            i.remove(j)
        
print(substrings)

[['ac', 'ct', 'ti', 'iv', 'vi', 'is', 'on', 'act', 'ion', 'vision'], ['or', 'rot', 'rota']]
